In [10]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 0

In [11]:
#load in the policies to the vectorstore
loaders = []
docs = []

#create a list with the names of the files to be loaded in the data/policies folder
for file in os.listdir('data/policies'):
    if file.endswith('.txt'):
        loader = TextLoader('data/policies/' + file)
        loaders.append(loader)

for loader in loaders:
    docs.extend(loader.load())

if times_embedding_run < 1:
    text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
    documents = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(openai_api_key=key)
    policies = Chroma.from_documents(documents, embeddings)
    times_embedding_run += 1

"Embeddings generated into policies variable"

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


'Embeddings generated into policies variable'

In [3]:
targets = Property('targets', None, "Any target pests or rodents that the customer is having issues with", required=True, examples=['cockroaches', 'rats', 'mice'])

location_of_issue = Property('location_of_issue', None, "The location of the issue", required=True, examples=['kitchen', 'bathroom', 'bedroom', 'proactive'])

service_type = Property('service_type', None, "The type of service the customer is looking for", required=True, examples=['extermination', 'inspection', 'proactive treatment'])

severity = Property('severity', None, "The severity of the issue. Can be low, medium or high", required=True, examples=['low', 'medium', 'high'])

special_instructions = Property('special_instructions', None, "Any special instructions for the technician", required=False, examples=['please call before arriving', 'please leave a note on the door'])

time_constraints = Property('time_constraints', None, "Any time constraints for the technician", required=False, examples=['please arrive between 9am and 11am', 'please arrive between 1pm and 3pm'])

issues = NaturalLanguagePool([targets, location_of_issue, service_type, severity, special_instructions, time_constraints], "The issues the customer is having", create_new_properties=True)

In [6]:
chat = ChatOpenAI(openai_api_key=key)

initial_prompt = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases."""

history = DynamicChatHistory(chat, system_message=initial_prompt, additional_context="", initial_history=[SystemMessage(content="Hi! My name is George from Better Termite & Pest Control. How can I help you? :)")])

stage_one = ConversationScriptStage('Take lead of conversation', "Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.", lambda: any(isinstance(message, AIMessage) for message in history.history))

stage_two = ConversationScriptStage('Get contact information', "Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.", lambda: customer_profile.is_complete(), pool=customer_profile, show_missing=True, show_complete=True)

stage_three = ConversationScriptStage('Get customer issue', "Get the customer's issue. This goal is considered complete, once you have their issue, severity, location of issue, service type, special instructions and time constraints.", lambda: issues.is_complete(), pool=issues, show_missing=True, show_complete=True)

sales_script = ConversationScript([stage_one, stage_two, stage_three])

agent = Conversation(history, chat, sales_script, customer_profile, policies)

In [8]:
agent.respond('Hi name is John Smith and I am interested in pest control')

Agent:  Hello John! Thank you for reaching out to Better Termite & Pest Control. May I ask a few questions to better understand your pest control needs?


In [9]:
agent.respond('Yeah sure, but before we start, do you offer squirrel control?')

Agent:  I'm sorry, we do not offer any wildlife control services, including control for squirrels. Our pest control services mainly focus on insect and rodent control. However, we can definitely assist you with any pest issues you may be having in your home.


In [10]:
agent.respond('No worries. I also have an ant problem in my kitchen')

Agent:  We could definitely assist you with that! To better assist you, may I have your phone number, email, street address, city, state, and zip code, please? That way, I can have one of our representatives get in touch with you as soon as possible.


In [11]:
agent.respond('Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878')

Agent:  Thank you for providing that information George! I would be happy to help you with your ant problem. Can you tell me more about the ants? Where are you seeing them in your home? Are they appearing in any specific areas or rooms? Also, do you have any specific targets or areas that you're concerned about? Lastly, are you looking for a one-time service or recurring service?


In [14]:
agent.customer_profile.data

[first_name: George,
 last_name: Schulz,
 phone_number: 571-431-9531,
 email: georgeschulz33@gmail.com,
 street_address: 123 Elm Way,
 city: Gaithersburg,
 state: MD,
 zip: 20878]